In [44]:
import pandas as pd
import pickle

with open("/home/Papafrita/Projects/brd4_binding/data/" +
            "out_dict_200k.pickle", "rb") as file:
    data_dict = pickle.load(file)

data = pd.DataFrame.from_dict(data_dict, orient="index")

200000

In [45]:
data = data.set_index(data.columns[0])
data = data.rename_axis(index=['id'])

In [46]:
from ast import literal_eval

for col in data.columns:
    data[col] = data[col].apply(literal_eval)

: 

In [25]:
binds = data[data["binds"] == 1].shape[0]
no_binds = data[data["binds"] == 0].shape[0]
print("Bind/No Bind ratio: {:.2f}".format(binds/no_binds))

Bind/No Bind ratio: 0.00


In [26]:
data.head()

,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,SPS,MolWt,HeavyAtomMolWt,ExactMolWt,NumValenceElectrons,...,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea,binds
id,,,,,,,,,,,,,,,,,,,,,
221249733,"(10.542537792894935, 5.510416666666666, 11.566...","(10.542537792894935, 5.510416666666666, 11.566...","(0.03935185185185164, 0.3414351851851849, 0.01...","(-1.3236111111111108, 0.3414351851851849, -3.0...","(0.629627805420395, 0.5706569889658594, 0.7134...","(9.23076923076923, 36.666666666666664, 24.2307...","(182.135, 129.203, 197.25900000000001)","(176.08699999999996, 114.08299999999998, 186.1...","(182.032756672, 129.1153641, 197.051049592)","(68, 54, 70)",...,"(0, 0, 0)","(0, 0, 1)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)",0
221249736,"(10.542537792894935, 5.510416666666666, 5.7922...","(10.542537792894935, 5.510416666666666, 5.7922...","(0.03935185185185164, 0.3414351851851849, 0.55...","(-1.3236111111111108, 0.3414351851851849, 0.55...","(0.629627805420395, 0.5706569889658594, 0.8414...","(9.23076923076923, 36.666666666666664, 22.8666...","(182.135, 129.203, 222.357)","(176.08699999999996, 114.08299999999998, 204.2...","(182.032756672, 129.1153641, 222.119069576)","(68, 54, 82)",...,"(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)",0
221249880,"(10.542537792894935, 5.510416666666666, 5.2421...","(10.542537792894935, 5.510416666666666, 5.2421...","(0.03935185185185164, 0.3414351851851849, 0.56...","(-1.3236111111111108, 0.3414351851851849, 0.56...","(0.629627805420395, 0.5706569889658594, 0.5803...","(9.23076923076923, 36.666666666666664, 9.28571...","(182.135, 129.203, 114.17299999999997)","(176.08699999999996, 114.08299999999998, 108.125)","(182.032756672, 129.1153641, 114.02516919199999)","(68, 54, 38)",...,"(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 1)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)",0
221250369,"(10.542537792894935, 11.667083333333334, 10.73...","(10.542537792894935, 11.667083333333334, 10.73...","(0.03935185185185164, 0.02027777777777784, 0.0)","(-1.3236111111111108, -3.1709027777777785, -0....","(0.629627805420395, 0.6907794287953419, 0.6451...","(9.23076923076923, 30.5, 8.0)","(182.135, 222.30999999999995, 218.12800000000001)","(176.08699999999996, 204.16599999999997, 200.992)","(182.032756672, 222.103813436, 217.07486752399...","(68, 84, 76)",...,"(0, 1, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 1)",0
221250546,"(10.542537792894935, 11.667083333333334, 10.81...","(10.542537792894935, 11.667083333333334, 10.81...","(0.03935185185185164, 0.02027777777777784, 0.2...","(-1.3236111111111108, -3.1709027777777785, 0.2...","(0.629627805420395, 0.6907794287953419, 0.5134...","(9.23076923076923, 30.5, 27.555555555555557)","(182.135, 222.30999999999995, 128.175)","(176.08699999999996, 204.16599999999997, 116.0...","(182.032756672, 222.103813436, 128.094963004)","(68, 84, 52)",...,"(0, 1, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)","(0, 0, 0)",0


In [27]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, LSTM, Dense, GlobalMaxPooling1D, Input


In [28]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [29]:
from sklearn.model_selection import train_test_split

feature_cols = data.columns[:-1].to_list()
X = data[feature_cols]
y = data.binds

In [30]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [39]:
(data.iloc[0, 0])

'(10.542537792894935, 5.510416666666666, 11.56675925925926)'

In [ ]:
def create_cnn_model(input_shape):
    model = Sequential(
        Input(shape=input_shape),
        Conv1D(64, kernel_size=3, activation="relu"),
        Conv1D(64, kernel_size=3, activation="relu"),
        GlobalMaxPooling1D(),
        Dense(64, activation="relu"),
        Dense(1, activation="sigmoid"),
        )
